In [ ]:
import math
# https://en.wikipedia.org/wiki/B%C3%A9zier_curve#Quadratic_curves

import array, calendar, json, random, time

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return "Point(%s,%s)" % (self.x, self.y)        

    def __add__(self, p):
        return Point(self.x + p.x, self.y + p.y)    

    def __sub__(self, p):
        return Point(self.x - p.x, self.y - p.y)    
    
    def __mul__(self, scalar):        
        return Point(self.x * scalar, self.y * scalar)    

    def distance(self, p):
        return math.sqrt((p.x-self.x)*(p.x-self.x) + (p.y-self.y)*(p.y-self.y))

    def get_slope(self, p):
        m = (p.y - self.y)/(p.x - self.x)
        return m
    
    def get_point_on_line(self,p,t):
        return (p - self) * t + self
    
def get_control_point(t, pt, p0, p2):
    #return p1*(1-t)*2 + p0*2*(1-t)*t + p2*t*2
    return pt*2 - p0*0.5 - p2*0.5


def get_d_point(d, m, p):
    x = p.x + d / (math.sqrt(1+m*m))
    y = p.y + d*m / (math.sqrt(1+m*m))
    return Point(x,y) 

    
def get_perpendicular_slope(m):
    return -1.0/m
    
    
def get_xy(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]


def get_epoch(datestr, formatstr):
    return calendar.timegm(time.strptime(datestr, formatstr))


def build_point(org, dst, epoch0, epoch1):
    p0 = Point(org['properties']['webmercator'][0], org['properties']['webmercator'][1])
    p2 = Point(dst['properties']['webmercator'][0], dst['properties']['webmercator'][1])
    d = math.sqrt(p0.distance(p2))*0.5
    m = get_perpendicular_slope(p0.get_slope(p2))
    p_mid = get_d_point(random.uniform(-1*d,d),m,p0.get_point_on_line(p2,0.5))
    p1 = get_control_point(0.5, p_mid, p0, p2)
    return [p0.x,p0.y,p1.x,p1.y,p2.x,p2.y,epoch0,epoch1]

def build_point2(org, dst, epoch0, epoch1):
    p0 = Point(org['properties']['webmercator'][0], org['properties']['webmercator'][1])
    p2 = Point(dst['properties']['webmercator'][0], dst['properties']['webmercator'][1])
    d = math.sqrt(p0.distance(p2))
    m = get_perpendicular_slope(p0.get_slope(p2))
    space = random.uniform(d*0.55,d)
    if p0.y > p2.y:
        space *= -1.0
    p_mid = get_d_point(space,m,p0.get_point_on_line(p2,0.5))
    p1 = get_control_point(0.5, p_mid, p0, p2)
    return [p0.x,p0.y,p1.x,p1.y,p2.x,p2.y,epoch0,epoch1]

In [ ]:
with open("../data/making-a-point-flow-map/country-centroids.geojson") as f:
    data = json.load(f)

In [ ]:
# Example 1 
random.seed(0)
N = 25
origins = random.sample(data['features'], N)
destinations = random.sample(data['features'], N)
values = [ random.randrange(100,5000) for _ in range(N)]

start_epoch = get_epoch('2020-01-01', '%Y-%m-%d')
end_epoch = get_epoch('2021-01-01', '%Y-%m-%d')
points = []
for i in range(0, N):
    org = origins[i]
    dst = destinations[i]
    val = values[i]
    for j in range(0,val):
        epoch0 = random.randrange(start_epoch, end_epoch, 24*60*60)
        epoch1 = random.randrange(start_epoch, end_epoch, 24*60*60)
        if epoch0 > epoch1:
            point = build_point(org,dst,epoch1,epoch0)
        else:
            point = build_point(org,dst,epoch0,epoch1)
        points.append(point)
random.shuffle(points)
flatlist = []
for point in points:
    flatlist += point
array.array('f', flatlist).tofile(open('bezier-example1.bin', 'wb'))


!rsync bezier-example1.bin YOUR_HOST


In [ ]:
print("TRUE	making_a_point_flow_map_bezier_example1	Making EarthTime Maps	Bezier 1	CREATE Lab	bdrk		20200101	20210101	1	https://PATH_TO_/bezier-example1.bin			point-flow																		8	WebGLVectorTile2.pointFlowVertexShader	WebGLVectorTile2.pointFlowFragmentShader	WebGLVectorTile2.prototype._drawPointFlow")

In [ ]:
#Example 2
random.seed(4)
N = 25
origins = random.sample(data['features'], N)
destinations = random.sample(data['features'], N)
values = [ random.randrange(100,10000) for _ in range(N)]

start_epoch = get_epoch('2020-01-01', '%Y-%m-%d')
end_epoch = get_epoch('2022-01-01', '%Y-%m-%d')
points = []
for i in range(0, N):
    org = origins[i]
    dst = destinations[i]
    val = values[i]
    for j in range(0,val):
        epoch0 = random.randrange(start_epoch, end_epoch, 24*60*60)
        epoch1 = random.randrange(start_epoch, end_epoch, 24*60*60)
        if epoch0 > epoch1:
            point = build_point2(org,dst,epoch1,epoch0)
        else:
            point = build_point2(org,dst,epoch0,epoch1)
        points.append(point)
random.shuffle(points)
flatlist = []
for point in points:
    flatlist += point
array.array('f', flatlist).tofile(open('bezier-example2.bin', 'wb'))
!rsync bezier-example2.bin YOUR_HOST


In [ ]:
print('TRUE	making_a_point_flow_map_bezier_example2	Making EarthTime Maps	Bezier 2	CREATE Lab	bdrk		20200101	20220101	1	https://PATH_TO/bezier-example2.bin			point-flow																		8	WebGLVectorTile2.pointFlowVertexShader	WebGLVectorTile2.pointFlowFragmentShader	WebGLVectorTile2.prototype._drawPointFlow')